In [86]:
#!curl -fsSL https://ollama.com/install.sh | sh

In [87]:
# RUN IN TWO SEPARATE TERMINALS
#!ollama serve
#!ollama run llama3

In [88]:
# LangChain supports many other chat models. Here, we're using Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


chat = ChatOllama(model="llama3")
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")


chain = prompt | chat | StrOutputParser()


print(chain.invoke({"topic": "Space travel"}))

Why did the astronaut break up with his girlfriend before going to Mars?

Because he needed space! (get it?)


In [89]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("The_Three-Body_Problem.pdf")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=32000, chunk_overlap=1000)
all_splits = text_splitter.split_documents(data)

In [90]:
from langchain_chroma import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings

model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings(model_name=model_name, gpt4all_kwargs=gpt4all_kwargs))

In [91]:
question = "What is the name of the planet surrounded by three suns?"
docs = vectorstore.similarity_search(question, k = 10)
len(docs)

10

In [92]:
docs[0].page_content

'“No. There are three!” The minister cried, tears flowing from his face.\n“The other two are behind that one!”\nEveryone stared at the sun again, still confused.\n“The Im perial Observatory has confirmed that right now we are\nexperiencing th e extremely rare phenomenon of a tri-solar syzygy . The\nthree suns are i n a straight line, moving around our planet at the same\nangular speed! Thus, our planet and the three suns are in a straight line with\nour world at the end!”'

In [93]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context. 
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

In [94]:
from langchain_core.messages import AIMessage, HumanMessage

query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
        ),
    ]
)

query_transformation_chain = query_transform_prompt | chat

query_transformation_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Can LangSmith help test my LLM applications?"),
            AIMessage(
                content="Yes, LangSmith can help test and evaluate your LLM applications. It allows you to quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced costs. Additionally, LangSmith can be used to monitor your application, log all traces, visualize latency and token usage statistics, and troubleshoot specific issues as they arise."
            ),
            HumanMessage(content="Tell me more!"),
        ],
    }
)

AIMessage(content='"LangSmith LLM application testing"', response_metadata={'model': 'llama3', 'created_at': '2024-07-23T08:57:43.164204321Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 461605553, 'load_duration': 16475556, 'prompt_eval_count': 146, 'prompt_eval_duration': 66231000, 'eval_count': 9, 'eval_duration': 108323000}, id='run-b32dee62-2779-4b40-8471-1574319a9f14-0')

In [95]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

retriever = vectorstore.as_retriever(k=25)


query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 1,
        # If only one message, then we just pass that message's content to retriever
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    # If messages, then we pass inputs to LLM chain to transform the query, then pass to retriever
    query_transform_prompt | chat | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

In [96]:
from typing import Dict

from langchain_core.runnables import RunnablePassthrough


def parse_retriever_input(params: Dict):
    return params["messages"][-1].content


In [97]:
SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context. 
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

In [103]:
conversational_retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="What is the main technology of the red coast base?"),
        ]
    }
)["answer"]

'According to the context, the main technology of Red Coast Base is its transmission power, which was as high as twenty-five megawatts! This was achieved through the use of a set of gigantic capacitors.'

In [104]:
conversational_retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="What is the name of the particle sent to Earth by Trisolaris to block scientific progress?"),
        ]
    }
)["answer"]

'According to the context, the particles sent to Earth by Trisolaris are called "sophons".'

In [106]:
conversational_retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="What is the name of the virtual reality game the protagonist joins?"),
        ]
    }
)["answer"]

'The name of the virtual reality game that the protagonist joins is "Three Body".'

In [107]:
conversational_retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Give some names of characters that play the game called three body"),
        ]
    }
)["answer"]

'According to the context, the following are names of characters mentioned in relation to the game "Three-Body":\n\n1. King Wen of Zhou\n2. Mozi\n\nThese characters appear in the novel\'s Part II: Three Body.'